In [1]:
import os
from langchain.callbacks import wandb_tracing_enabled


os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_PROJECT"] = "langchain-tracing"

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Tags(BaseModel):
    sentiment: str = Field(..., enum=["happy", "sad", "neutral", "angry"])
    aggressiveness: int = Field(...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    tone: str = Field(..., 
        enum=["formal", "informal", "friendly", "assertive", "humorous", "sympathetic", "authoritative", "neutral"]
    )

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.chains.structured_output.base import create_structured_output_runnable


llm = ChatOpenAI(temperature=1)
structured_llm = create_structured_output_runnable(
    Tags,
    llm,
    mode="openai-tools",
    enforce_function_usage=True,
    return_single=True
)
tagger_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are world class sentence tagger"),
        ("human", "tag the given sentence:\n\n{sentence}"),
    ]
)
tagger_chain = (tagger_prompt | structured_llm)

In [4]:
from operator import itemgetter

prompt2 = ChatPromptTemplate.from_messages([("human", "rephrase the sentence using this tag information:\n\ntag: {tag}\n\nsentence: {sentence}")])

chain2 = (
    {"tag": tagger_chain, "sentence": itemgetter("sentence")}
    | prompt2
    | llm
)

In [5]:
from langchain.callbacks.tracers import ConsoleCallbackHandler


# Now, we unset the environment variable and use a context manager.
if "LANGCHAIN_WANDB_TRACING" in os.environ:
    del os.environ["LANGCHAIN_WANDB_TRACING"]

# enable tracing using a context manager
with wandb_tracing_enabled():
    response = chain2.invoke({
            "sentence": "And then a puppy came to his life and showed him what it means to love and be loved.",
        }, 
        config={'callbacks': [ConsoleCallbackHandler()]}
    )

import json

json.loads(response.json())

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Streaming LangChain activity to W&B at https://wandb.ai/parth-singh71/langchain-tracing/runs/mlr5pmba
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.


[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "sentence": "And then a puppy came to his life and showed him what it means to love and be loved."
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<tag,sentence>] Entering Chain run with input:
{
  "sentence": "And then a puppy came to his life and showed him what it means to love and be loved."
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<tag,sentence> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "sentence": "And then a puppy came to his life and showed him what it means to love and be loved."
}[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<tag,sentence> > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "sentence": "And then a puppy came to his life and showed him what it means to love and be loved."
}

[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<tag,sentence> > 3:chain:RunnableSequence > 4:promp

wandb: WARNING WARNING: Failed to serialize model: 'int' object is not subscriptable


[llm/end] [1:chain:RunnableSequence > 8:llm:ChatOpenAI] [1.44s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The puppy entering his life brought him immense joy and taught him the true meaning of love and companionship. #happy #sympathetic",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The puppy entering his life brought him immense joy and taught him the true meaning of love and companionship. #happy #sympathetic",
            "additional_kwargs": {},
            "name": null,
            "id": null
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 28,
      "prom

{'content': 'The puppy entering his life brought him immense joy and taught him the true meaning of love and companionship. #happy #sympathetic',
 'additional_kwargs': {},
 'response_metadata': {'token_usage': {'completion_tokens': 28,
   'prompt_tokens': 55,
   'total_tokens': 83},
  'model_name': 'gpt-3.5-turbo',
  'system_fingerprint': 'fp_3bc1b5746c',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': None,
 'example': False}